In [ ]:
import numpy as np
import pandas as pd

# Parameters
S0 = 100          # Initial stock price
K = 110           # Strike price
T = 0.25          # Time to maturity in years
r = 0.02          # Risk-free rate
sigma = 0.30      # Volatility
c = 0.01          # Dividend yield
n = 15            # Number of binomial periods
u = 1.0395        # Up factor (given, unrounded in calculation)
d = 1 / u         # Down factor
dt = T / n        # Time step
R = np.exp(r * dt)  # Risk-free rate per step
disc = np.exp(-r * dt)  # Discount factor per step

# Risk-neutral probability
q = (np.exp((r - c) * dt) - d) / (u - d)

# Step 1: Build stock price tree
stock_tree = np.zeros((n+1, n+1))
for i in range(n+1):
    for j in range(i+1):
        stock_tree[j, i] = S0 * (u ** (i - j)) * (d ** j)

# Step 2: Build option price tree for American Call
option_tree = np.zeros_like(stock_tree)
exercise_tree = np.full_like(stock_tree, False, dtype=bool)

# Terminal payoff
for j in range(n+1):
    option_tree[j, n] = max(stock_tree[j, n] - K, 0)

# Backward induction
early_exercise_period = None
for i in range(n-1, -1, -1):
    for j in range(i+1):
        hold = disc * (q * option_tree[j, i+1] + (1 - q) * option_tree[j+1, i+1])
        exercise = stock_tree[j, i] - K
        option_tree[j, i] = max(hold, exercise)
        if exercise > hold:
            exercise_tree[j, i] = True
            early_exercise_period = i  # last update will be the earliest optimal period

# Collect final result
american_call_price = option_tree[0, 0]

# Prepare output
stock_df = pd.DataFrame(stock_tree)
option_df = pd.DataFrame(option_tree)
exercise_df = pd.DataFrame(exercise_tree)

american_call_price, early_exercise_period, stock_df.round(2), option_df.round(4), exercise_df

: 